In [1]:
from fetchbim.admin import get_family
from fetchbim.bimservice import get_ids
from fetchbim.family import Family

In [2]:
family_ids = get_ids()

In [ ]:
for family_id in family_ids:
    fam_dict = get_family(family_id)
    props = fam_dict.get('Properties')
    fam = Family.from_json(fam_dict)
    response = fam.post_notion()
    print(response.json())